In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pylab as plt
from fbprophet import Prophet
import matplotlib.pyplot as pplt

In [ ]:
final_data = pd.read_csv("./final_data.csv")
final_data_interpolated = pd.read_csv("./final_data_interpolated.csv")
final_data.loc[:,"Date"] =  pd.to_datetime(final_data["Date"])
final_data_interpolated.loc[:,"Date"] =  pd.to_datetime(final_data_interpolated["Date"])

In [ ]:
#sets cutoff date for training data as March 1st
split_date = "2019-03-01"
df_test = final_data
df_test = df_test[df_test["Date"] >= dt.datetime.strptime("2000-01-01",'%Y-%m-%d')]
df_test = df_test[df_test["Date"] < dt.datetime.strptime(split_date,'%Y-%m-%d')]
train = df_test
train = train.rename(columns={'Date': 'ds', 'Count': 'y'})

In [ ]:
m = Prophet(changepoint_prior_scale=0.05,yearly_seasonality=80, seasonality_mode='multiplicative')
m.add_regressor('GDDSUM_shift60',mode='additive')
#m.add_regressor('PRCP_shift60',mode='additive')
m.fit(train)

In [ ]:
#Creates forecast for 60 days after March 1st
future = m.make_future_dataframe(periods=60)
future = future.merge(final_data, left_on = 'ds', right_on = 'Date')
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
forecast_error = forecast.merge(final_data, left_on = 'ds', right_on = 'Date')[['ds', 'Count','yhat','Doy']]

In [ ]:
forecast_error = forecast_error[forecast_error["ds"] >= dt.datetime.strptime("2019-03-01",'%Y-%m-%d')]
forecast_error = forecast_error[forecast_error["ds"] <= dt.datetime.strptime("2019-05-01",'%Y-%m-%d')]

In [ ]:
forecast_error['sq_error'] = (forecast_error.Count - forecast_error.yhat)**2

In [ ]:
#trims off days outside of peak pollen season
test_length_forecast = forecast_error.loc[(forecast_error["Doy"]<=119) & (forecast_error["Doy"]>59),"sq_error" ]
fb_rmse = np.sqrt(test_length_forecast.mean())
fb_rmse

In [ ]:
pplt.plot(forecast_error.ds, forecast_error.Count, '-', label='Count')
pplt.plot(forecast_error.ds, forecast_error.yhat, '-', label='Forecast')
pplt.legend(loc='best')
pplt.show()

In [ ]:
#Creates dataframe with years as columns
simple_predictor = final_data.pivot(index='Doy', columns='Year', values='Count')
simple_predictor.columns

In [ ]:
#Calculates average 'naive' predictor and error for that predictor
simple_predictor['avg_predictor_20'] = simple_predictor.drop(columns = [2019]).mean(axis=1)
simple_predictor['sq_error_20'] = (simple_predictor.avg_predictor_20-simple_predictor[2019])**2

In [ ]:
#trims off days outside of peak pollen season
test_length_simple = simple_predictor.loc[(simple_predictor.index<=119) & (simple_predictor.index>59),"sq_error_20" ]
fb_simp_rmse = np.sqrt(test_length_simple.mean())
fb_simp_rmse

In [ ]:

pplt.plot(forecast_error.Doy, forecast_error.Count, '-', label='Count')
pplt.plot(forecast_error.Doy, forecast_error.yhat, '-', label='Forecast')
pplt.plot(simple_predictor.index, simple_predictor.avg_predictor_20, '-', label='Simple')
pplt.legend(loc='best')
pplt.show()

In [ ]:
  def individual_prediction_comparison_plot(forecast_error, simple_predictor, first_day, last_day):
    forecast_error = forecast_error.loc[forecast_error.Doy <= last_day ,:]
    forecast_error = forecast_error.loc[forecast_error.Doy >= first_day ,:]
    simple_predictor = simple_predictor.loc[simple_predictor.index <= last_day ,:]
    simple_predictor = simple_predictor.loc[simple_predictor.index >= first_day ,:]
    
    pplt.plot(forecast_error.Doy, forecast_error.Count, '-', label='Count')
    pplt.plot(forecast_error.Doy, forecast_error.yhat, '-', label='Forecast')
    pplt.plot(simple_predictor.index[simple_predictor.index<=last_day], simple_predictor.avg_predictor_20[simple_predictor.index<=last_day], '-', label='Simple')
    pplt.legend(loc='best')
    pplt.xlabel("Day of Year")
    pplt.ylabel("Pollen Count")
    pplt.legend(title = "")
    pplt.show()

individual_prediction_comparison_plot(forecast_error, simple_predictor, 40, 130)